# To run this UI, pleas run all 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

out = widgets.Output()
result_output = widgets.Output()
debug_out = widgets.Output()

In [3]:
widgets.VBox([out, result_output])

In [4]:
import io

In [5]:
tab = widgets.Tab()

dirty_file_upload = widgets.FileUpload(
    accept=".csv",
    multiple=False
)

cleaned_text_output_label = widgets.Label(
    "Output File Name:", layout=widgets.Layout(width='150px')
)

cleaned_text_output_filename = widgets.Text(
    value="cleaned.csv",
    layout=widgets.Layout(width='150px')
)
clean_button = widgets.Button(
    description="Normalize data",
    icon='play',
    tooltip='Normalize data'
)


clean_file_upload = widgets.FileUpload(
    accept=".csv",
    multiple=False
)
start_button = widgets.Button(
    description='Generate docx',
    icon="play",
    tooltip='Generate docx',
)


In [6]:
def label_and_input(label, label_layout=None, text_default_value=None, input_layout=None):

    if label_layout is None:
        label_layout = widgets.Layout(min_width='150px')
    else:
        label_layout.min_width = '150px'
    if text_default_value is None:
        text_default_value = ""
    if input_layout is None:
        input_layout = widgets.Layout()

    return widgets.HBox([
        widgets.Label(label, layout=label_layout),
        widgets.Text(value=text_default_value)
    ])


def label_and_vertical_input(label, label_layout=None, text_default_value=None, input_layout=None):

    if label_layout is None:
        label_layout = widgets.Layout(min_width='150px')
    else:
        label_layout.min_width = '150px'
    if text_default_value is None:
        text_default_value = ""
    if input_layout is None:
        input_layout = widgets.Layout()

    return widgets.VBox([
        widgets.Label(label, layout=label_layout),
        widgets.Text(value=text_default_value, input_layout=input_layout)
    ])


def label_and_textarea(label, label_layout=None, text_default_value=None, input_layout=None):

    if label_layout is None:
        label_layout = widgets.Layout(min_width='150px')
    else:
        label_layout.min_width = '150px'
    if text_default_value is None:
        text_default_value = ""
    if input_layout is None:
        input_layout = widgets.Layout()

    return widgets.VBox([
        widgets.Label(label, layout=label_layout),
        widgets.Textarea(value=text_default_value, input_layout=input_layout)
    ])

In [7]:
def get_label_and_something_value(x):
    return x.children[1].value

In [8]:
# add a label for as a status bar 
generate_staus_bar = widgets.Label("")

In [9]:
generated_output_filename = label_and_input("Output Filename:", text_default_value="demo.docx")

initial_name_tags = widgets.VBox([
    label_and_input("First Name", text_default_value='First Name'),
    label_and_input("Middle Initials", text_default_value='Middle Initial(s)'),
    label_and_input("Last Name", text_default_value="Last Name")
])

initial_examples = widgets.VBox([
label_and_input("Initials for \"Xiang-Zhen\": ", text_default_value="X-Z"),
label_and_input("Initials for \"Jun Soo\": ", text_default_value="J-S"),
label_and_input("Initials for \"Baskin-Sommers\": ", text_default_value="B-S"),
label_and_input("Initials for \"van Rooij\": ", text_default_value="vR")
])

generate_column_names = widgets.VBox([
    label_and_input("Full Name: ", text_default_value="Compound Name + highest degree"),
    label_and_input("Role(s): ", text_default_value="Role(s)"),
    label_and_textarea("Affiliations Information (as a Python List of List):", 
                      text_default_value="[['Affiliation 1 Department, Institution', 'City (e.g. Brisbane)', 'State', 'Country']]",
                       input_layout=widgets.Layout(width='600px')),
])

generate_contribution_priority_order = widgets.VBox([
    label_and_textarea("Roles priority: ", input_layout=widgets.Layout(height="200%"), text_default_value="[]")
])
accordion = widgets.Accordion([initial_name_tags, initial_examples, generate_column_names, generate_contribution_priority_order])
accordion.set_title(0, "Column names for reading authors' name")
accordion.set_title(1, "Format of initials from giving examples")
accordion.set_title(2, "Column names for generating authorship documentation")
accordion.set_title(3, "Roles priority")
accordion.selected_index = None

In [10]:
# this is data-cleaning part: to normalize email label

duplicate_email_output = widgets.Output()

open_refine_go = widgets.Button(
    description="Run OpenRefine",
    icon="play",
    tooltip="Run OpenRefine",
)
open_refile_status = widgets.Label("")

open_refine_tab = widgets.VBox([
    dirty_file_upload,
    label_and_input("Refined Output Filename", text_default_value="authors_refined.csv"),
    label_and_vertical_input("Column names for (Country, City, State)  to reconcile",
                            text_default_value="""[['City', 'State', 'Country']]"""),
    widgets.HBox([open_refine_go, open_refile_status])
])

email_go = widgets.Button(
    description = "Fund Duplicate",
    icon="play",
    tooltip="Find Duplicate"
)
email_status = widgets.Label("")
email_upload = widgets.FileUpload()
email_column_tags = widgets.VBox([
    email_upload, 
    label_and_input("Column Name for Email Address", text_default_value='Email Address'),
    widgets.HBox([email_go, email_status])
])
#

affiliation_go = widgets.Button(
    description = "Guess"
)
affiliation_status = widgets.Label()
affiliation_upload = widgets.FileUpload()
coi_input =  label_and_input("Disclousre Tag Name", text_default_value="Conflict of Interest/Disclosures")
funding_input = label_and_input("Funding Tag Name", text_default_value="Funding Acknowledgements")

generate_affiliations_tags = widgets.VBox([
    affiliation_upload,
    label_and_input("Output filename: ", text_default_value="authors_reference.csv"),
    widgets.Label("Indicies indicating Affiliation Information (indicating [Department, Street, City, Zip, Country] for each sublist)"),
    widgets.Textarea(value="""[[11, 12, 13, 14, 16],[17,18, 19, 20, 22],[23, 24, 25, 26, 28]]""", layout=widgets.Layout(width='600px')),
    coi_input,
    funding_input,
    widgets.HBox([affiliation_go, affiliation_status])
])

In [11]:

accordion_for_cleaning = widgets.Accordion([open_refine_tab, email_column_tags, generate_affiliations_tags])

accordion_for_cleaning.set_title(0, "Use OpenRefine for location naming matching")
accordion_for_cleaning.set_title(1, "Find duplciate entry")
accordion_for_cleaning.set_title(2, "Guess the department names")

accordion_for_cleaning.selected_index = None

In [12]:
from sklearn.base import TransformerMixin


class StringCleaner(TransformerMixin):

    def transform(self, X, **transform_params):
        return ["None" if type(s) is not str else s for s in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}


# %%

from spacy.lang.en import English
import string

parser = English()
punctuation = string.punctuation


def spacy_tokenizer(sentence):
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in parser(sentence)]
    tokens = [token for token in tokens if not (token in punctuation)]

    return tokens

In [13]:
from annotator import *

In [14]:
@debug_out.capture()
def run_openrfine(b):
    from openrefine_caller import openrefine_reconcile, create_project

    mydict = dirty_file_upload.value
    bytes_val = mydict[next(iter(mydict.keys()))]['content']

    ff = io.BytesIO(bytes_val)
    df_dirty = pd.read_csv(ff)
    df_dirty.to_csv("temp.csv")
    
    open_refile_status.value = "Calling OpenRefine..."

    output_filename = get_label_and_something_value(open_refine_tab.children[1])
    column_tags = eval(get_label_and_something_value(open_refine_tab.children[2]))
    
    try: 
        project_id = create_project("temp.csv")
        open_refile_status.value = "Running Reconcilation, check http://localhost:3333/project?project=" + project_id
        openrefine_reconcile("temp.csv", column_tags, output_csv=output_filename, project_id=project_id)
    except Exception as e:
        open_refile_status.value = "Something wrong happen, please check http://localhost:3333/project?project=" + project_id
        return 
    
    open_refile_status.value = "Finished, the output has been written to %s" % output_filename
    
            

# call function in the project address_linkage.py
@debug_out.capture()
def email_deduplicate(b):
    import io
    from generate_docx1 import InitialsGenerator, DocGenerator
    from address_linkage import EmailChecking
    
    result_output.clear_output()
    mydict = email_upload.value
    bytes_val = mydict[next(iter(mydict.keys()))]['content']

    ff = io.BytesIO(bytes_val)
    df_dirty = pd.read_csv(ff)
    
    email_check = EmailChecking()
    email_check.email_tags = email_column_tags.children[1].children[1].value
    duplicated_email = email_check.transform(df_dirty)
    
    with result_output:
        print("Possible duplicate authors (ordered by email address): ")
        for email_address, group in duplicated_email.groupby(email_check.email_tags):
            display(widgets.HTML("<h3> %s</h3>" % email_address))
            display(group)

    email_status.value = "Finshed! Please check the following log seection for duplicate rows"
    
@debug_out.capture()
def propose_data(b):
    from another_record_linkage import DepartmentNameNormalizer
    from annotator import add_coi_and_funding_prediction, StringCleaner, spacy_tokenizer
            
    mydict = affiliation_upload.value
    bytes_val = mydict[next(iter(mydict.keys()))]['content']
    if len(mydict) == 0:
        affiliation_status.value = "Error: there is no file uploaded"
        return 
    
    ff = io.BytesIO(bytes_val)
    df = pd.read_csv(ff)
    
    output_filename = get_label_and_something_value(generate_affiliations_tags.children[1])
    indices = eval(generate_affiliations_tags.children[3].value)
    
    normalizer = DepartmentNameNormalizer()
    normalizer.address_column_indices = indices
    
    df_referred = normalizer.give_reference(df)

    
    coi_tag = get_label_and_something_value(coi_input)
    funding_tag = get_label_and_something_value(funding_input)
    df_referred = add_coi_and_funding_prediction(df_referred, coi_tag=coi_tag, funding_tag=funding_tag)
    
    df_referred.to_csv(output_filename)
    affiliation_status.value = "Finished! The output has been written in %s" % output_filename
    


In [15]:
@debug_out.capture()
def start_generation(b):
    
    debug_out.clear_output()
    generate_staus_bar.value = "Generating..."
    
    import io
    from generate_docx1 import InitialsGenerator, DocGenerator
    from utils import AddressCombinator
    mydict = clean_file_upload.value
    if len(mydict) == 0:
        generate_staus_bar.value = "Error: there is no file uploaded"
    else:
        bytes_val = mydict[next(iter(mydict.keys()))]['content']

        f = io.BytesIO(bytes_val)
        df = pd.read_csv(f)

        initials_generator: InitialsGenerator = InitialsGenerator()
        initials_generator.first_name_tag = initial_name_tags.children[0].children[1].value
        initials_generator.middle_initial_tag = initial_name_tags.children[1].children[1].value
        initials_generator.last_name_tag = initial_name_tags.children[2].children[1].value

        initials_generator.initials_examples = {
            "Xiang-Zhen": get_label_and_something_value(initial_examples.children[0]),
            "Jun Soo": get_label_and_something_value(initial_examples.children[1]),
            "Baskin-Sommers": get_label_and_something_value(initial_examples.children[2]),
            "van Rooij": get_label_and_something_value(initial_examples.children[3])
        }

        initials =  initials_generator.transform(df)
        doc_generator = DocGenerator()
        #FIXME: find a better way of input array
        doc_generator.whole_name = get_label_and_something_value(generate_column_names.children[0])
        doc_generator.role_tag = generate_column_names.children[1].children[1].value
        
        affiliation_information = eval(get_label_and_something_value(generate_column_names.children[2]))
        doc_generator.affiliation_tags = [x[0] for x in affiliation_information] 
        
        for one_list in affiliation_information:
            address_combinator = AddressCombinator()
            address_combinator.tags = one_list 
            df[one_list[0]] = df.apply(address_combinator.combine_address, axis=1)
        
        doc_generator.output_doc_filename = get_label_and_something_value(generated_output_filename)
        doc_generator.roles_priority = eval(get_label_and_something_value(generate_contribution_priority_order.children[0]))
        doc_generator.generate(df, initials['Initial'])

        generate_staus_bar.value = "Finished! The document was generated at %s"% doc_generator.output_doc_filename

In [16]:
# add on_click, JSON to be determined
start_button.on_click(start_generation)
email_go.on_click(email_deduplicate)
open_refine_go.on_click(run_openrfine)
affiliation_go.on_click(propose_data)

generate_configuration = widgets.Textarea(
    description="Configuration JSON"
)

In [17]:
clean_tab = widgets.VBox([
    accordion_for_cleaning
])
generate_tab = widgets.VBox([
    widgets.HBox([clean_file_upload, start_button, generate_staus_bar], layout=widgets.Layout(margin="10px")),
    generated_output_filename,
    accordion
])
tab.children = [clean_tab, generate_tab]
tab.set_title(0, "Data Cleaning")
tab.set_title(1, "Generate Docx")

In [18]:
title = widgets.HTML("<h1>Authorship Statement Auto Generator</h1>")
tab.layout=widgets.Layout(width="80%", height="450px", min_width="800px")
generate_staus_bar.value = ""
with out:
    out.clear_output()
    display(        
        widgets.VBox([
            title,
            tab,
        ], layout=widgets.Layout(align_items="center"))
    )

result_output.clear_output()
with result_output:
    display(widgets.HTML("<h2>Log</h2>"))
    
debug_out.clear_output()
with debug_out:
    display(widgets.HTML("<h2>Debug output</h2>"))

In [19]:
debug_out

Output()